# TriAPI Smoke Test (gpt-5_2025-08-07)

最小可运行示例：获取 bearer token，调用 TriAPI（gcr/shared, gpt-5_2025-08-07, api-version 2025-02-01-preview）。
- scope 默认 `api://trapi/.default`，如果仍 403，可改为 `https://trapi.research.microsoft.com/.default` 再试。
- 身份来源：优先 AZURE_OPENAI_AD_TOKEN 环境变量，否则 Azure CLI，其次 Managed Identity（client id 默认 b32444ac-27e2-4f36-ab71-b664f6876f00）。


In [ ]:
import os, textwrap
from azure.identity import AzureCliCredential, ManagedIdentityCredential, ChainedTokenCredential, get_bearer_token_provider
from openai import AzureOpenAI

# 可改参数
SCOPE = os.environ.get("TRIAPI_SCOPE", "api://trapi/.default")
INSTANCE = os.environ.get("TRIAPI_INSTANCE", "gcr/shared")
DEPLOYMENT = os.environ.get("TRIAPI_DEPLOYMENT", "gpt-5_2025-08-07")
API_VERSION = os.environ.get("AZURE_API_VERSION", "2025-02-01-preview")
MI_CLIENT_ID = os.environ.get("TRIAPI_MI_CLIENT_ID", "b32444ac-27e2-4f36-ab71-b664f6876f00")

print(f"scope={SCOPE}
instance={INSTANCE}
deployment={DEPLOYMENT}
api_version={API_VERSION}
mi_client_id={MI_CLIENT_ID}")


In [ ]:
# 获取 token（优先 env，fallback 到 CLI + MI）
_env_token = os.environ.get("AZURE_OPENAI_AD_TOKEN")
if _env_token:
    token_provider = lambda: _env_token
    print(f"Using AZURE_OPENAI_AD_TOKEN from env (len={len(_env_token)})")
else:
    cred = ChainedTokenCredential(
        AzureCliCredential(),
        ManagedIdentityCredential(client_id=MI_CLIENT_ID),
    )
    token_provider = get_bearer_token_provider(cred, SCOPE)
    token = token_provider()
    print(f"Fetched token via ChainedCredential (len={len(token)})")
    os.environ["AZURE_OPENAI_AD_TOKEN"] = token  # 方便后续使用


In [ ]:
# 调用 TriAPI
endpoint = f"https://trapi.research.microsoft.com/{INSTANCE}".rstrip("/")
client = AzureOpenAI(
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
    api_version=API_VERSION,
)

try:
    resp = client.chat.completions.create(
        model=DEPLOYMENT,
        messages=[{"role": "user", "content": "ping"}],
        max_completion_tokens=8,
        # 温度不要改，默认 1；部分模型不接受 0/其他值。
    )
    content = resp.choices[0].message.content
    print("=== call success ===")
    print(content)
    print(resp)
except Exception as exc:  # noqa: BLE001
    print("=== call failed ===")
    print(repr(exc))
    raise
